In [1]:
%load_ext autoreload
%autoreload 2

import time
import mrpt
import numpy as np
from scipy import spatial
from sklearn.preprocessing import scale
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors

from gtda.plotting import plot_diagram
from steenroder import *
import gudhi

from matplotlib import pyplot as plt

from utils import get_complex, get_density_filtration, get_density_filtration_efficient, get_persistence

In [2]:
patches = np.load("./../data/patches.npy")
print(patches.shape)
points = patches.reshape(patches.shape[0]*patches.shape[1], -1)

(4167, 1000, 8)


## Sklearn NNs

In [9]:
leaf_size = 1000
k = 100
p = 10

In [4]:
nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto', leaf_size=leaf_size,
                        n_jobs=10, )

In [5]:
nbrs.fit(points)

NearestNeighbors(leaf_size=1000, n_jobs=10, n_neighbors=100)

In [7]:
distances, neighbors = nbrs.kneighbors(points)
## this cell took ~1hr on the Dell -> run on the cluster next time?

In [20]:
kth_distance = distances[:, k-1]
kth_index = np.argsort(kth_distance)[:len(distances)//p]
x_k_p = points[kth_index, :]

In [21]:
np.save("./../data/M_100_10.npy", x_k_p)

### Compute $Q$

In [4]:
patches = np.load("./../data/M_100_10.npy")

points = patches
q_bar_new_basis = np.load("./../data/Q_bar_big_neighborhood.npy")
closest_indices = np.argmin(spatial.distance.cdist(q_bar_new_basis, points), axis=1)
closest_points = points[closest_indices]

np.save("./../data/Q_from_M_100_10.npy", closest_points)

## Sklearn NNs with cosine

In [3]:
leaf_size = 1000
k = 100
p = 10

In [4]:
nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto', leaf_size=leaf_size,
                        n_jobs=10, metric="cosine")

In [5]:
nbrs.fit(points)

NearestNeighbors(leaf_size=1000, metric='cosine', n_jobs=10, n_neighbors=100)

In [6]:
distances, neighbors = nbrs.kneighbors(points)
## this cell took ~1hr on the Dell -> run on the cluster next time?

KeyboardInterrupt: 

In [ ]:
kth_distance = distances[:, k-1]
kth_index = np.argsort(kth_distance)[:len(distances)//p]
x_k_p = points[kth_index, :]

In [ ]:
np.save("./../data/M_100_10_cosine.npy", x_k_p)

## Approximate

In [ ]:
k = 100
target_recall = 0.9

In [4]:
points = points.astype('float32')

In [8]:
index = mrpt.MRPTIndex(points)

index.build_autotune_sample(target_recall, k+1)

# Don't run this anymore! index.save(f"./../data/mrpti_{k}_index")

In [9]:
print(index.ann(points[0:3], return_distances=False))

[[      0 1520014 3756022 3445013  375494   46443  387167 3894172 2951342
  1000002 1966042 1394011 1501246 1560088  209299  825119  129520 2494009
  1665034 1268233 1930008  917244 1928114  387416  251016 2616114 3925414
  3269092 2908067 3503252 3294021 4118002 2578195 3815025 3068507 3401209
  2063021 2780616 2920191 3120214  355285 2554065 1853008 2394034  674120
   780311 1558104  969735 2434021 2381694 1675230 1258193 2039078 1960338
  2263226  124514 3311122  463147 1877040 3669023 1928110 1338005  387273
  3938498 1654030 3853021  620013 2941006 3894026 3473576 1810288 3389190
  3057062 2514030   10006 3374209 3488011  830127   57222  135216 1675290
  1982075 3863031 2970508 3306047 1982072  433031 3651048  126048 3272337
  1123232 2653299 1527182 2686114   73751 1153359 1589378  360115 2617165
  2853083  402195]
 [      1  519441 3012003  885151 3821020 2208264 2518244 3151419 3607039
  3083131 1760103 1833815 3415064 1694365 1520029 2970656  837951 3420053
   387352  402161  

In [5]:
index = mrpt.MRPTIndex(points)

index.load(f"./../data/mrpti_{k}_index")

In [6]:
kth_neighbors = index.ann(points, return_distances=False)[:, k]

In [7]:
np.save("./../data/M_100_10_neighbors.npy", kth_neighbors)

In [15]:
v = points - points[kth_neighbors]
distances = np.sum(v**2, axis=1)
top_10_indices = np.argsort(distances)[:len(distances)//10]
m_100_10 = points[top_10_indices]

In [18]:
np.save("./../data/M_100_10.npy", m_100_10)